<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_provaPostProcessing_Federica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
@File name: model1_cross_validation.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import math

import numpy as np

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Activation
from keras.callbacks import EarlyStopping

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('local_modules')

#import postprocessing_sw
#import postprocessing_sw_HL
import plotting

  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built liac-arff


Using TensorFlow backend.


In [0]:
#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
'''
@Description: generate a multilayer perceptron with LeakyRelu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 1
#REGOLA: input/2, input, 2*input, 1
#layers TUTTE LE FEATURE: 57, 113, 226, 1 
#layers TIME CO2 TEMP: 21, 41, 82, 1 
#layers TIME CO2 TEMP PM25/TVOC: 30, 59, 118, 1 
#layers TIME CO2 TEMP PM25 TVOC: 39, 77, 154, 1 

def generate_model_leaky(shape, n_features):
  
  units_1 = int(n_features/2)
  units_2 = n_features
  units_3 = n_features*2
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model


'''
@Description: generate a multilayer perceptron with Relu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 2
#REGOLA: a= input, b= a*2/3+c, c= b*2/3+1
#layers TUTTE LE FEATURE: 113, 229, 153, 1
#layers TIME CO2 TEMP: 41, 85, 57, 1 
#layers TIME CO2 TEMP PM25/TVOC: 59, 121, 81, 1 
#layers TIME CO2 TEMP PM25 TVOC: 77, 157, 105, 1 

def generate_model(shape, n_features):
  
  a = np.array([[1,0,0],[-(2/3),1,-1],[0,-(2/3),1]])
  b = np.array([n_features,0,1])
  x = np.linalg.solve(a, b)
  
  units_1 = int(x[0])
  units_2 = int(x[1])
  units_3 = int(x[2])
  
  model = Sequential()
 
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [5]:
#@title SCEGLI I PARAMETRI

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS
loss = 'binary_crossentropy'

#MODEL
modello = 1 #@param {type:"integer"}

#DEFINE K-FOLD CROSS-VALIDATION
fold = 10 #@param {type:"integer"}

kfold = KFold(n_splits=fold, shuffle=False, random_state=None)

cvscores = []
predictions = []
true_p =[]
true_n = []
test_time =[]
real = []

dimSplit = math.floor(len(dataset[:,0])/10)
startIndex = 0
finishIndex = dimSplit-1

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  print("\nGenerate model...")

  if modello==1 :
    model = generate_model_leaky(X[train].shape[-1], n_features)
  elif modello ==2:
    model = generate_model(X[train].shape[-1], n_features)

  #COMPILE MODEL
  print("\nCompile model...")
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])

  #FIT MODEL
  print("\nFit model...")
  epoche = 7 #@param{type:"integer"}
  history = model.fit(X[train], Y[train], epochs=epoche, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  print("\nEvaluate model...")
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  print("\nCalculate predictions...")
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  flat_pred = [item for sublist in pred for item in sublist]
  predictions.append(flat_pred)
  real.append(Y[test])
  
  #STORE DATETIME
  time = []  
  for j in X[test]:
    time.append(int(j[-5]))  
  test_time.append(time)
  
  #PLOTTING
  #plotting.plot_model_results(history)
  #plotting.plot_co2_temp_cross(flat_pred, dimSplit, startIndex, finishIndex, i)
  
  #CONFUSION MATRIX
  print("\nCompute confusion matrix...")
  y_true = Y[test]
  tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
  (tn, fp, fn, tp)
  other = 100*tn/(tn+fp)
  pasto = 100*tp/(fn+tp)
  true_p.append(pasto)
  true_n.append(other)
  print("Other: %.2f %%" % other)
  print("Pasto: %.2f %%" % pasto)
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("MEAN TRUE POSITIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_p), np.std(true_p)))
print("MEAN TRUE NEGATIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_n), np.std(true_n)))


Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE
Instructions for updating:
Colocations handled automatically by placer.

FOLD: 1

Generate model...

Compile model...

Fit model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/7
315441/315441 [==============================] - 15s 49us/step - loss: 0.5214 - acc: 0.7736
Epoch 2/7
315441/315441 [==============================] - 14s 44us/step - loss: 0.4153 - acc: 0.8365
Epoch 3/7
315441/315441 [==============================] - 14s 45us/step - loss: 0.3907 - acc: 0.8334
Epoch 4/7
315441/315441 [==============================] - 14s 46us/step - loss: 0.3817 - acc: 0.8296
Epoch 5/7
315441/315441 [==============================] - 14s 44us/step - loss: 0.3751 - acc: 0.8292
Epoch 6/7
315441/315441 [==============================] - 15s 47us/step - loss: 0.3687 - acc: 0.8294
Epoch 7/7
315441/315441 

In [6]:
flat_predictions = [item for sublist in predictions for item in sublist]
true_y = [arr.tolist() for arr in real]
flat_true_y = [item for sublist in true_y for item in sublist]
flat_time = [item for sublist in test_time for item in sublist]
'''
t0 = test_time[0]
t1 = test_time[1]
t = []
t.append(t0)
t.append(t1)
flat_t = [item for sublist in t for item in sublist]
print(len(flat_t))
print(len(t0))
print(flat_time[0:35049])
print(flat_t[0:35049])
print(flat_time[35049:70098])
print(flat_t[35049:70098])
'''

'\nt0 = test_time[0]\nt1 = test_time[1]\nt = []\nt.append(t0)\nt.append(t1)\nflat_t = [item for sublist in t for item in sublist]\nprint(len(flat_t))\nprint(len(t0))\nprint(flat_time[0:35049])\nprint(flat_t[0:35049])\nprint(flat_time[35049:70098])\nprint(flat_t[35049:70098])\n'

In [7]:
#CONFUSION MATRIX
print("\nCompute confusion matrix...")
tn, fp, fn, tp = confusion_matrix(flat_true_y, flat_predictions).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)


Compute confusion matrix...
TN 289894
FP 44273
FN 3519
TP 12805
Other corretti: 86.75 %
Pasto corretti: 78.44 %


In [0]:
import more_itertools
import numpy

def sliding_windows(flat_pred, window_len):
  
  print("\nFUNZIONE SLIDING WINDOWS...")
  print("Input: " +str(len(flat_pred)))
 
  half_window = int(window_len/2)
  
  windowsList = list(more_itertools.windowed(flat_pred,n=window_len, step=1))

  result = [];
  result.append(flat_pred[0:half_window])
  i = 0 
  while i < len(windowsList): 
    count = 0
    c = windowsList[i][half_window]

    j = 0  
    while j < window_len:
      if windowsList[i][j] == c :
        count = count + 1
      j = j + 1

    if count<(half_window+2) :
      if c == 1:
        result.append(0)
      else:
        result.append(1)
    else:
      result.append(c)

    i = i + 1

  result.append(flat_pred[-(half_window+1):-1])

  flat_result = numpy.hstack(result)
  
  print("RISULTATO...")
  print(len(flat_result))

  return flat_result


def sliding_windows_time_colazione(new_pred_1, time, window_len):
  
  print("\nFUNZIONE SLIDING WINDOWS TIME COLAZIONE...")
  print("Input: " +str(len(new_pred_1)))
  
  colazione = []
  result = []
  
  cont1 = 0
  cont2 = 0
  
  half_window = int(window_len/2)
  
  for l,t in zip(new_pred_1, time):
    
    if(t>=360 and t<=630):
      colazione.append(l)
      cont1+=1
      
      if((t+1)>630):
        #print(len(colazione))
        
        if(len(colazione)>window_len):
          windowsList_colazione = list(more_itertools.windowed(colazione,n=window_len, step=1))

          result.append(colazione[0:half_window])
          i = 0 
          while i < len(windowsList_colazione): 
            count = 0
            c = windowsList_colazione[i][half_window]

            j = 0  
            while j < window_len:
              if windowsList_colazione[i][j] == c :
                count = count + 1
              j = j + 1

            if count<(half_window+2) :
              if c == 1:
                result.append(0)
              else:
                result.append(1)
            else:
              result.append(c)

            i = i + 1

          result.append(colazione[-(half_window+1):-1])

          del colazione[:]
          
        else:
          result.append(colazione)
        
    else:
      result.append(l)
      cont2+=1
  
  flat_result = numpy.hstack(result)
  #print(list(result))
  #print(list(flat_result))
  print("RISULTATO...\n(colazione/altro/somma/risultato/risultato flat)")
  print(cont1, cont2, (cont1+cont2), len(result), len(flat_result))

  return flat_result
        

def sliding_windows_time_pranzocena(new_pred_2, time, window_len_p, window_len_c):
  
  print("\nFUNZIONE SLIDING WINDOWS TIME PRANZO CENA...")
  print("Input: " +str(len(new_pred_2)))
  
  pranzo = []
  cena = []
  result = []
  
  cont1 = 0
  cont2 = 0
  cont3 = 0

  half_window_p = int(window_len_p/2)
  half_window_c = int(window_len_c/2)
  
  for l,t in zip(new_pred_2, time):
    
    if(t>=690 and t<=900):
      pranzo.append(l)
      cont2+=1
      
      if((t+1)>900):
        
        if(len(pranzo)>window_len_p):
          windowsList_pranzo = list(more_itertools.windowed(pranzo,n=window_len_p, step=1))

          result.append(pranzo[0:half_window_p])
          i = 0 
          while i < len(windowsList_pranzo): 
            count = 0
            c = windowsList_pranzo[i][half_window_p]

            j = 0  
            while j < window_len_p:
              if windowsList_pranzo[i][j] == c :
                count = count + 1
              j = j + 1

            if count<(half_window_p+2) :
              if c == 1:
                result.append(0)
              else:
                result.append(1)
            else:
              result.append(c)

            i = i + 1

          result.append(pranzo[-(half_window_p+1):-1])

          del pranzo[:]
      
        else:
          result.append(pranzo)
         
    elif(t>=1080 and t<=1400):
      cena.append(l)
      cont3+=1

      if((t+1)>1400):
        
        if(len(cena)>window_len_c):
          windowsList_cena = list(more_itertools.windowed(cena,n=window_len_c, step=1))

          result.append(cena[0:half_window_c])
          i = 0 
          while i < len(windowsList_cena): 
            count = 0
            c = windowsList_cena[i][half_window_c]

            j = 0  
            while j < window_len_c:
              if windowsList_cena[i][j] == c :
                count = count + 1
              j = j + 1

            if count<(half_window_c+2) :
              if c == 1:
                result.append(0)
              else:
                result.append(1)
            else:
              result.append(c)

            i = i + 1

          result.append(cena[-(half_window_c+1):-1])

          del cena[:]
        
        else:
          result.append(cena)
          
    else:
      result.append(l)
      cont1+=1

  flat_result = numpy.hstack(result)
  
  print("RISULTATO...\n(pranzo/cena/altro/somma/risultato/risultato flat")
  print(cont2, cont3, cont1, (cont1+cont2+cont3),len(result),len(flat_result))

  return flat_result




In [153]:
'''
new_pred_1 = postprocessing_sw.sliding_windows(postprocessing_sw.sliding_windows(pred,5),5)
new_pred_2 = postprocessing_sw.sliding_windows_time_colazione(postprocessing_sw.sliding_windows_time_colazione(new_pred_1, time,5), time, 11)
new_pred_3 = postprocessing_sw.sliding_windows_time_pranzocena(postprocessing_sw.sliding_windows_time_pranzocena(new_pred_2, time, 31, 31),time, 61, 61)
new_pred_4 = postprocessing_sw.sliding_windows(postprocessing_sw.sliding_windows(new_pred_3,15),15)
'''
  
new_pred_1 = sliding_windows(sliding_windows(flat_predictions,5),9)
new_pred_2 = sliding_windows_time_colazione(sliding_windows_time_colazione(new_pred_1, flat_time,5), flat_time,5)
new_pred_3 = sliding_windows_time_pranzocena(sliding_windows_time_pranzocena(new_pred_2, flat_time, 15, 41),flat_time, 25, 51)
new_pred_4 = sliding_windows(new_pred_3,11)

new_pred = new_pred_4



FUNZIONE SLIDING WINDOWS...
Input: 350491
RISULTATO...
350491

FUNZIONE SLIDING WINDOWS...
Input: 350491
RISULTATO...
350491

FUNZIONE SLIDING WINDOWS TIME COLAZIONE...
Input: 350491
RISULTATO...
(colazione/altro/somma/risultato/risultato flat)
66569 283922 350491 350010 350491

FUNZIONE SLIDING WINDOWS TIME COLAZIONE...
Input: 350491
RISULTATO...
(colazione/altro/somma/risultato/risultato flat)
66569 283922 350491 350010 350491

FUNZIONE SLIDING WINDOWS TIME PRANZO CENA...
Input: 350491
RISULTATO...
(pranzo/cena/altro/somma/risultato/risultato flat
51419 77543 221529 350491 338598 350491

FUNZIONE SLIDING WINDOWS TIME PRANZO CENA...
Input: 350491
RISULTATO...
(pranzo/cena/altro/somma/risultato/risultato flat
51419 77543 221529 350491 333818 350491

FUNZIONE SLIDING WINDOWS...
Input: 350491
RISULTATO...
350491


In [154]:

#CONFUSION MATRIX
print("\nCompute NEW confusion matrix...")
tn, fp, fn, tp = confusion_matrix(flat_true_y, new_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
 
plotting.plot_co2_temp_complete_dataset(flat_predictions,new_pred)


Compute NEW confusion matrix...
TN 291720
FP 42447
FN 4038
TP 12286
Other corretti: 87.30 %
Pasto corretti: 75.26 %


In [0]:
'''
cont_p_tot = 0
cont_p1 = 0
cont_p2 = 0

cont_c_tot = 0
cont_c1 = 0
cont_c2 = 0

cont_a_tot = 0

window_len = 31

pranzo = []
cena = []

for time, pred in zip(test_time, predictions):

  for p,t in zip(pred, time):

    if(t>=631 and t<=900):
        pranzo.append(p)
        cont_p_tot += 1

        if(len(pranzo)>=window_len):
          cont_p1 += 1
          del pranzo [:]
    elif(t<631 or (t>900 and t<1110)):
      cont_a_tot += 1
    
  if(len(pranzo) != 0  and len(pranzo)<window_len):# and (pranzo[-1]!=900 or pranzo[0]!=631)):
      #print("LA DIMENSIONE DI PRANZO E' INFERIORE A WINDOW_LEN")
      cont_p2 += 1
      del pranzo [:]

  for p,t in zip(pred, time):
    
    if(t>=1110 and t<=1379):
        cena.append(p)
        cont_c_tot += 1

        if(len(cena)>=window_len):
          cont_c1 += 1
          del cena [:]
    elif(t>1379):
         cont_a_tot += 1
          
  if(len(cena)!=0 and len(cena)<window_len):# and (cena[-1]!=1379 or cena[0]!=1110)):
      #print("LA DIMENSIONE DI CENA E' INFERIORE A WINDOW_LEN")
      cont_c2 += 1
      del cena [:]
        

  print(cont_p1, cont_p2, cont_p_tot)
  print(cont_c1, cont_c2, cont_c_tot)
  print(cont_a_tot)
  print(cont_p_tot+cont_c_tot+cont_a_tot)
  
  cont_p_tot = 0
  cont_p1 = 0
  cont_p2 = 0

  cont_c_tot = 0
  cont_c1 = 0
  cont_c2 = 0

  cont_a_tot = 0
'''

In [0]:
'''
def sliding_windows_time_colazione(new_pred_1, time, window_len):
  
  #print("\nFUNZIONE SLIDING WINDOWS TIME COLAZIONE...")
  #print("Input: " +str(len(new_pred_1)))
  
  colazione = []
  result = []
  
  cont1 = 0
  cont2 = 0
  
  half_window = int(window_len/2)
  
  for l,t in zip(new_pred_1, time):
    
    if(t>=300 and t<=630):
      colazione.append(l)
      cont1+=1
      
      if(len(colazione)==window_len):

        result.append(colazione[0:half_window])
        count=0
        c = colazione[half_window]

        j = 0  
        while j < window_len:
          if colazione[j] == c :
            count = count + 1
          j = j + 1

        if count<(half_window+2) :
          if c == 1:
            result.append(0)
          else:
            result.append(1)
        else:
          result.append(c)

        result.append(colazione[-(half_window+1):-1])

        del colazione[:]
        
    else:
      result.append(l)
      cont2+=1
  
        
  if(len(colazione) != 0  and len(colazione)<window_len):
    #print("LA DIMENSIONE DI COLAZIONE E' INFERIORE A WINDOW_LEN")
    #print(len(colazione))
    result.append(colazione)

  
  flat_result = numpy.hstack(result)
  
  
  #print("RISULTATO...\n(cont colazione/cont altro/somma cont/risultato/risultato flat)")
  #print(cont1, cont2, (cont1+cont2), len(result),len(flat_result))

  return flat_result
        

def sliding_windows_time_pranzo(new_pred_2, time, window_len_p):
  
  #print("\nFUNZIONE SLIDING WINDOWS TIME PRANZO CENA...")
  #print("Input: " +str(len(new_pred_2)))
  
  pranzo = []
  result = []
  
  cont1 = 0
  cont2 = 0
  
  half_window_p = int(window_len_p/2)
  
  for l,t in zip(new_pred_2, time):
    
    if(t>=631 and t<=900):
      pranzo.append(l)
      cont2+=1
      
      if(len(pranzo)==window_len_p):

        result.append(pranzo[0:half_window_p])
        count = 0
        c = pranzo[half_window_p]

        j = 0 

        while j < window_len_p:
          if pranzo[j] == c :
            count = count + 1
          j = j + 1

        if count<(half_window_p+2) :
          if c == 1:
            result.append(0)
          else:
            result.append(1)
        else:
          result.append(c)

        result.append(pranzo[-(half_window_p+1):-1])

        del pranzo[:]
      
    else:
      result.append(l)
      cont1+=1
      
  if(len(pranzo) != 0  and len(pranzo)<window_len_p):
    #print("LA DIMENSIONE DI PRANZO E' INFERIORE A WINDOW_LEN")
    #print(len(pranzo))
    result.append(pranzo)
  
  flat_result = numpy.hstack(result)
  #print(list(result))
  #print(list(flat_result))
  #print(cont2, cont1, (cont2+cont1), len(result), len(flat_result))

  return flat_result
    
def sliding_windows_time_cena(new_pred_2, time, window_len_c):
  
  cena = []
  result = []
  
  cont1 = 0
  cont3 = 0
  
  half_window_c = int(window_len_c/2)
  
  for l,t in zip(new_pred_2, time):
    
    if(t>=1110 and t<=1379):
      cena.append(l)
      cont3+=1

      if(len(cena)==window_len_c):

        result.append(cena[0:half_window_c])
        count = 0
        c = cena[half_window_c]

        j = 0  
        while j < window_len_c:
          if cena[j] == c :
            count = count + 1
          j = j + 1

        if count<(half_window_c+2) :
          if c == 1:
            result.append(0)
          else:
            result.append(1)
        else:
          result.append(c)

        result.append(cena[-(half_window_c+1):-1])
        
        del cena[:]
        
    else:
      result.append(l)
      cont1+=1
      
  if(len(cena) != 0  and len(cena)<window_len_c):
    #print("LA DIMENSIONE DI CENA E' INFERIORE A WINDOW_LEN")
    #print(len(cena))
    result.append(cena)
      
  flat_result = numpy.hstack(result)

  return flat_result
'''